In [0]:
#Importing necessary libraries

from IPython import display
import math
import string
from pprint import pprint
from num2words import num2words
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [0]:
pip install praw

In [0]:
pip install num2words

In [0]:
#client id- StmWjOzTODd8Og
#user agent- r/webscraper
#secret- 3-HWmn6SD7a32SEoOLXyVSUv5jQ

import praw
reddit = praw.Reddit(client_id='StmWjOzTODd8Og', client_secret='3-HWmn6SD7a32SEoOLXyVSUv5jQ', user_agent='r/webscraper')

In [0]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [0]:
#Storing the titles of extracted posts into a set to avoid repetition

posts = set()

for post in reddit.subreddit('gameofthrones').hot(limit=500):
  posts.add(post.title)

print(posts)
print(len(posts))

{'[No Spoilers] Game of Thrones tattoo that I got earlier this year, Fresh vs Healed.', "[NO SPOILERS] Wouldn't wanna be near them at the gas station.", '[Spoilers] Squeaky Daenerys says Dracrays', "[NO SPOILERS] I am not sure if anyone here would be interested but I make GoT's mods and here is the trailer for my latest :)", '[Spoilers] There Ain’t Nothing Like A Dame No. 2 (Dame Diana in Gourd, 2019)', '[SPOILERS] Did Robb really need to make that deal with Walder Frey?', '[No Spoilers] Throwback: Arya with Ghost', '[NO SPOILERS] I did GOT as an old text based videogame from the 80s. Pixelart Timelapse FANART', "[No Spoilers] Miniature King's Landing Made out of Cardboard!", '[NO SPOILERS] "Power Is Power" A Fan Art by Ertac Altinoz', '[NO SPOILERS] Here’s an unfinished pointillism portrait I did of Tyrion. Because “art is never finished only abandoned”.', '[NO SPOILERS] Winterfell 3D concept model according to the books by Shadiversity', "[NO SPOILERS] Who did robert's bastard's kill

In [0]:
#Brute sentiment analysis without preprocessing and cleaning of extracted data

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for post in posts:
  score = sia.polarity_scores(post)
  score['post'] = post
  results.append(score)
  
pprint(results[:5], width=100)

[{'compound': 0.3182,
  'neg': 0.0,
  'neu': 0.85,
  'pos': 0.15,
  'post': '[No Spoilers] Game of Thrones tattoo that I got earlier this year, Fresh vs Healed.'},
 {'compound': 0.0,
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0,
  'post': "[NO SPOILERS] Wouldn't wanna be near them at the gas station."},
 {'compound': 0.0,
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0,
  'post': '[Spoilers] Squeaky Daenerys says Dracrays'},
 {'compound': 0.6563,
  'neg': 0.052,
  'neu': 0.741,
  'pos': 0.206,
  'post': "[NO SPOILERS] I am not sure if anyone here would be interested but I make GoT's mods "
          'and here is the trailer for my latest :)'},
 {'compound': -0.0572,
  'neg': 0.151,
  'neu': 0.714,
  'pos': 0.135,
  'post': '[Spoilers] There Ain’t Nothing Like A Dame No. 2 (Dame Diana in Gourd, 2019)'}]


In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
#Stopword removal, punctuation elimination and tokenization of extracted data

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

remove = set(stopwords.words('english'))
filtered = []

for post in posts:
  #Converting numbers to words and removing punctuations to give bag of words
  post = [num2words(x) if x.isnumeric() else x for x in post.translate(str.maketrans('', '', string.punctuation)).lower().split()]
  
  #Removing 
  temp = [x for x in post if x not in remove]
  filtered.append(" ".join(temp).replace('spoilers', '').strip())

In [51]:
#Results of sentiment analysis after preprocessing

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for post in filtered:
  score = sia.polarity_scores(post)
  score['post'] = post
  results.append(score)
  
pprint(results[:5], width=100)
data = pd.DataFrame.from_records(results)
data.head()

[{'compound': 0.3182,
  'neg': 0.0,
  'neu': 0.777,
  'pos': 0.223,
  'post': 'game thrones tattoo got earlier year fresh vs healed'},
 {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'post': 'wouldnt wanna near gas station'},
 {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'post': 'squeaky daenerys says dracrays'},
 {'compound': 0.6124,
  'neg': 0.0,
  'neu': 0.583,
  'pos': 0.417,
  'post': 'sure anyone would interested make gots mods trailer latest'},
 {'compound': -0.2755,
  'neg': 0.161,
  'neu': 0.839,
  'pos': 0.0,
  'post': 'ain’t nothing like dame two dame diana gourd two thousand and nineteen'}]


,neg,neu,pos,compound,post
0,0.000,0.777,0.223,0.3182,game thrones tattoo got earlier year fresh vs ...
1,0.000,1.000,0.000,0.0000,wouldnt wanna near gas station
2,0.000,1.000,0.000,0.0000,squeaky daenerys says dracrays
3,0.000,0.583,0.417,0.6124,sure anyone would interested make gots mods tr...
4,0.161,0.839,0.000,-0.2755,ain’t nothing like dame two dame diana gourd t...
